In [8]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# The overview of data retrieved from arxiv 

arxiv.py interact with arxiv api

## Library

* [arxiv.py](https://github.com/lukasschwab/arxiv.py)

In [2]:
array1 = '42 19 10 95 44 25 28 59 11 98 49 43 84 24 36 90 14 20 4 76 89 48 64 3 27 23 52 61 9 6 7 21 53 33 82 50 100 96 93 35 51 5 57 75 38 47 92 67 87 8 68 54 69 1 91 65 34 97 94 74 88 70 39 85 58 66 73 60 37 55 29 22 31 71 45 2 81 56 17 40 72 77 79 78 15 30 86 13 41 18 46 32 62 16 12 83 26 80 63 99' 

In [3]:
array2 = '88 72 59 23 35 33 21 27 48 91 78 73 70 12 99 54 55 16 24 31 96 44 67 22 5 85 51 62 74 60 61 77 87 8 81 19 86 25 40 76 2 89 43 41 9 92 80 97 34 83 32 26 20 82 90 17 39 30 69 6 52 36 45 4 79 65 7 57 37 66 95 49 75 1 13 28 53 42 100 94 46 47 11 58 84 68 14 18 98 63 29 64 93 56 38 50 3 15 10 71' 

In [15]:
array3 = '80 65 95 70 37 63 54 59 84 46 42 71 60 28 75 49 35 81 11 38 15 83 66 22 93 30 29 45 57 33 40 55 36 43 25 96 73 87 31 79 6 1 50 47 41 4 39 32 3 53 100 76 16 34 12 72 69 92 18 27 98 5 78 61 68 26 74 48 8 9 20 56 99 58 23 51 86 94 64 19 13 21 62 67 90 85 24 44 77 82 14 10 2 91 52 17 7 89 97 88'

In [17]:
len(set(array1.split(" "))) == 100
len(set(array2.split(" "))) == 100
len(set(array3.split(" "))) == 100


True

True

True

In [18]:
from requests.exceptions import HTTPError
from urllib.parse import quote_plus
from urllib.parse import urlencode
from urllib.request import urlretrieve
import feedparser
import re

In [ ]:
root_url = 'http://export.arxiv.org/api/'

In [ ]:
def query(search_query="",
          id_list=[],
          prune=True,
          start=0,
          max_results=10,
          sort_by="relevance",
          sort_order="descending"):
    url_args = urlencode({
        "search_query": search_query,
        "id_list": ','.join(id_list),
        "start": start,
        "max_results": max_results,
        "sortBy": sort_by,
        "sortOrder": sort_order
    })
    results = feedparser.parse(root_url + 'query?' + url_args)
    # results is a atom feed atomxxx.xml. Might be a dictionary, what will  feedparser.parse(xxxurl) return
    if results.get('status') != 200:
        # TODO: better error reporting
        raise Exception("HTTP Error " +
                        str(results.get('status', 'no status')) + " in query")
    else:
        results = results['entries']
    results = [r for r in results if r.get("title", None)]
    for result in results:
        # Renamings and modifications
        mod_query_result(result)
        if prune:
            prune_query_result(result)
    return results

In [ ]:
query(id_list=["1707.08567"])

In [20]:
search_query=""
id_list=[],
prune=True,
start=0,
max_results=10,
sort_by="relevance",
sort_order="descending"
id_list=["1707.08567"]

In [22]:
url_args = urlencode({
        "search_query": search_query,
        "id_list": ','.join(id_list),
        "start": start,
        "max_results": max_results,
        "sortBy": sort_by,
        "sortOrder": sort_order
    })
url_args 

'search_query=&id_list=1707.08567&start=%280%2C%29&max_results=%2810%2C%29&sortBy=%28%27relevance%27%2C%29&sortOrder=descending'

In [ ]:
root_url + 'query?' + url_args

In [ ]:
results = feedparser.parse(root_url + 'query?' + url_args)
results.get('status')

In [ ]:
results = results['entries']
results = [r for r in results if r.get("title", None)]

In [ ]:
for result in results:
    # Renamings and modifications
    mod_query_result(result)
    if prune:
        prune_query_result(result)
return results

In [ ]:
def mod_query_result(result):
    # Useful to have for download automation
    result['pdf_url'] = None
    for link in result['links']:
        if 'title' in link and link['title'] == 'pdf':
            result['pdf_url'] = link['href']
    result['affiliation'] = result.pop('arxiv_affiliation', 'None')
    result['arxiv_url'] = result.pop('link')
    result['title'] = result['title'].rstrip('\n')
    result['summary'] = result['summary'].rstrip('\n')
    result['authors'] = [d['name'] for d in result['authors']]
    if 'arxiv_comment' in result:
        result['arxiv_comment'] = result['arxiv_comment'].rstrip('\n')
    else:
        result['arxiv_comment'] = None
    if 'arxiv_journal_ref' in result:
        result['journal_reference'] = result.pop('arxiv_journal_ref')
    else:
        result['journal_reference'] = None
    if 'arxiv_doi' in result:
        result['doi'] = result.pop('arxiv_doi')
    else:
        result['doi'] = None

In [ ]:
results[0].pop('arxiv_affiliation', 'None')

In [ ]:
def prune_query_result(result):
    prune_keys = ['updated_parsed',
                  'published_parsed',
                  'arxiv_primary_category',
                  'summary_detail',
                  'author',
                  'author_detail',
                  'links',
                  'guidislink',
                  'title_detail',
                  'tags',
                  'id']
    for key in prune_keys:
        try:
            del result['key']
        except KeyError:
            pass

In [ ]:
def slugify(obj):
    # Remove special characters from object title
    filename = '_'.join(re.findall(r'\w+', obj.get('title', 'UNTITLED')))
    # Prepend object id
    filename = "%s.%s" % (obj.get('pdf_url').split('/')[-1], filename)
    return filename

In [ ]:
def download(obj, dirpath='./', slugify=slugify):
    if not obj.get('pdf_url', ''):
        print("Object has no PDF URL.")
        return
    if dirpath[-1] != '/':
        dirpath += '/'
    path = dirpath + slugify(obj) + '.pdf'
    urlretrieve(obj['pdf_url'], path)
    return path

In [ ]:
paper = query(id_list=["1707.08567"])
download(paper) 

* [arxivpy](https://github.com/titipata/arxivpy)
* [sotawhat](https://github.com/chiphuyen/sotawhat)
* [arxiv-checker](https://github.com/adamdempsey90/arxiv-checker)
* [arxivscraper](https://github.com/Mahdisadjadi/arxivscraper)

In [ ]:
import arxiv

In [ ]:
paper = arxiv.query(id_list=["1707.08567"])

In [ ]:
type(paper) == list 
len(paper)
paper[0]

for i in paper:
    print(i)
    

Table: search_query field prefixes
* prefix	explanation
* ti	Title
* au	Author
* abs	Abstract
* co	Comment
* jr	Journal Reference
* cat	Subject Category
* rn	Report Number
* id	Id (use id_list instead)
* all	All of the above

Search the result, access and download. Tidy up the name of the files

In [ ]:
# Query for a paper of interest, then download
paper = arxiv.query(id_list=["1707.08567"])[0]
arxiv.download(paper)
# You can skip the query step if you have the paper info!
paper2 = {"pdf_url": "http://arxiv.org/pdf/1707.08567v1",
          "title": "The Paper Title"}
arxiv.download(paper2)

# Returns the object id
def custom_slugify(obj):
    return obj.get('id').split('/')[-1]

# Download with a specified slugifier function
arxiv.download(paper, slugify=custom_slugify)

In [ ]:
paper.get('title')
paper.get('authors')
paper.get('summary')

In [ ]:
paper = arxiv.query(id_list=["1707.08567"])[0]

# Retrieve the data of an author 

In [ ]:
DH = arxiv.query(search_query="au: Dieter Horns")
len(DH)

In [ ]:
DH[3]['affiliation']

In [ ]:
 'a'  in DH[3]['affiliation']

In [ ]:
DH[0]['published'] # date time format yyyy-MM-ddTHH:mm:ssZ

In [ ]:
len(DH[0]['authors'])

In [ ]:
if DH[0]['authors'][0]=='Dieter Horns':
    print(2)

In [ ]:
for DHevery in DH: 
    print(DHevery['title'])
    print(DHevery['authors'])
    #print(DHevery['summary'])

In [ ]:
DHpaper = arxiv.query(id_list= ["1309.3846"])[0]

In [ ]:
paper2 = {"pdf_url": "http://arxiv.org/pdf/1707.08567v1",
          "title": "The Paper Title"}
arxiv.download(paper2)

## Workflow

In [ ]:
DH = arxiv.query(search_query="au:D. AND au:Horns AND (cat:astro-ph OR cat:hep-ph OR cat:hep-ex OR cat:id_list=physics.ins-det OR cat:astro-ph.HE OR cat:astro-ph.IM OR cat:astro-ph.CO)", max_results= 50)  

In [ ]:
for DHevery in DH: 
    everyitem = {"pdf_url": DHevery["pdf_url"],
                 "title" : DHevery["title"]}
    arxiv.download(everyitem)

In [ ]:
!touch DH_index.md
with open('DH_index.md', 'w') as the_file:
    # the_file.write('## Dieter Horns as First author\n')
    for DHevery in DH: 
        # if DHevery['authors'][0]=='Dieter Horns':
        # if 'University of Hamburg'  in DHevery['affiliation']:
        # if len(DHevery['authors']) < 4:
            the_file.write('* **Title:** ' + DHevery["title"] + '\n')  
            the_file.write('\n')
            the_file.write('  **Published at:** ' + DHevery['published'] + '\n')
            the_file.write('\n')
            the_file.write('  **pdf_url:** ' + DHevery['pdf_url'] + '\n') 
            the_file.write('\n')
            the_file.write('  **Summary:** ' + DHevery["summary"] + '\n') 
    the_file.close()

In [ ]:
!mv DH_index.md *.pdf ~/Documents/GammaRay/DieterHorns/

Then you can draw the content of summary from the retrieved data, then read the summary and find the interesting paper and download it with the help of its `pdf_url`.

## How to determine the author is the only one

In [ ]:
DH = arxiv.query(search_query="au:Dieter Horns", max_results= 50) #Only pick out names containing Dieter
DH[26]['authors']

In [ ]:
DH = arxiv.query(search_query="au:D. AND au:Horns AND cat:astro-ph ", max_results= 50) 

`arxiv.query` works less satisfied than `arxiv.download`

## Fetch the content between a time interval

In [ ]:
DH[0]['published'] # date time format yyyy-MM-ddTHH:mm:ssZ

In [ ]:
import dateutil.parser

d = dateutil.parser.parse('2008-09-26T01:51:42.000Z')
print(d.strftime('%m/%d/%Y'))  #==> '09/26/2008'

In [ ]:
dateutil.parser.parse(DH[0]['published']).strftime('%m/%d/%Y')

In [ ]:
import datetime

In [ ]:
date_format = "%Y-%m-%dT%H:%M:%S.%fZ" 
datetime.datetime.strptime('2008-09-26T01:51:42.000Z', date_format)
datetime_object = datetime.datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')
datetime_object

## Store the data using pandas

Compare with the [result](https://arxiv.org/search/?query=+Dieter+Horns&searchtype=author&abstracts=show&order=-announced_date_first&size=50&start=50) from manually search on arxiv, the result is well displayed, ordered by announced date.

## With built-in library

Below we include code snippets for these languages that perform the bare minimum functionality - calling the api and printing the raw Atom results

In [ ]:
import urllib.request
url = 'http://export.arxiv.org/api/query?search_query=all:electron&start=0&max_results=1'
data = urllib.request.urlopen(url).read() 
data 